In [11]:
import cameratransform as ct
import drawSvg as draw
from math import sin, cos, pi, asin
import numpy as np
from opensimplex import OpenSimplex
import random

In [12]:

gen = OpenSimplex(seed=123)
default_cam = ct.Camera(ct.RectilinearProjection(image=(800, 800)),
                        ct.SpatialOrientation(pos_y_m=-5000, elevation_m=-1800, tilt_deg=115, roll_deg=-40))
def dome(r=240, n=360, squash=0.6, layers=40, cam=default_cam, n_lines = 39, inset=0.04,
        tdiff=3, line_l=2.9, n_tent=0, noise_r=1, noise_coeff=1, bg_color='black', line_color='#3366ff'):
    random.seed(123)
    canvas = draw.Drawing(800, 800, center='origin', displayInline=False)
    canvas.append(draw.Rectangle(0, 0, 800, 800, fill=bg_color))
    rings = []
    lines = []
    height = r * 1.34 * squash
    
    for z in np.arange(0, height, height/layers):
        ring = []
        layer_r = r*cos(asin((r-z/squash)/r))
        if layer_r < 1e-5:
            continue
        for i in range(n):
            theta = i/n*2*pi
            ring.append([
                layer_r * cos(theta),
                layer_r * sin(theta),
                z
            ])
        rings.append(cam.imageFromSpace(ring).flatten())
        special_layer_r = layer_r
    
    offsets = []
    for _ in range(n_lines):
        offsets.append(random.uniform(1/tdiff, 1))
    total = sum(offsets)
    curr, thetas = 0, []
    for o in offsets:
        thetas.append(curr * 2 * pi)
        curr += o / total
    
    inner_height = r * (1.34 * (1 - 4*inset)) * squash
    for j, theta in enumerate(thetas):
        gen = OpenSimplex(seed=123+j)
        line = []
        rs = []
        line_step = inner_height/n *  2
        for z in np.arange(0, inner_height, line_step):
            sink = max(20 - z, 0) **2 * .1
            layer_r = r*cos(asin((r-z/squash)/r))
            rs.append(layer_r)
            line.append([
                layer_r * cos(theta) * (1 - inset),
                layer_r * sin(theta) * (1 - inset),
                z + height * inset + sink
            ])
        dr = rs[-1] - rs[-2]
        lol = random.uniform(1, 1.1)
        zs = np.arange(inner_height, lol*inner_height * line_l, line_step)
        steps = len(zs)
        target = 0.5
        for i, z in enumerate(zs):
            interp = i / steps
            layer_r += (dr * (1 - interp) + target * interp) * line_step
            line.append([
                layer_r * cos(theta) * (1 - inset),
                layer_r * sin(theta) * (1 - inset),
                z + height * inset + sink
            ])
            theta += gen.noise2d(noise_r*cos(theta), noise_r*sin(theta)) * interp * noise_coeff
            theta %= 2*pi
        
        lines.append(cam.imageFromSpace(line).flatten())
    n_tent = 0
    if n_tent:
        tent_gen = OpenSimplex(seed=123 + t)
        dr = rs[-1] - rs[-2]
        target = 20
        zs = np.arange(z + line_step * 4 + 30, inner_height * 3, line_step*8)
        maxi = len(zs)
        base_theta_offset = theta_offset
        layer_r = special_layer_r + 10
        for i, z in enumerate(zs):
            theta_offset = base_theta_offset + tent_gen.noise2d(0, 0.1*i) * 0.04
            linterp = i / maxi
            layer_r += min((linterp*target + (1-linterp)*dr)*line_step, 2) - 0.4
            a = [
                layer_r * cos(theta - theta_offset),
                layer_r * sin(theta - theta_offset),
                z + 20
            ]
            
            b = [
                layer_r * cos(theta + theta_offset),
                layer_r * sin(theta + theta_offset),
                z + 20
            ]
            lines.append(cam.imageFromSpace([a,b]).flatten())
            
    dtheta = 0.3
    base_theta = 0
    n_lines = 5
    line_spread = 0.18
    zstep = 10
    sink = 40
    final_height = height * 4
    spire_lines = make_spire(base_theta+pi, dtheta/2, n_lines//2, line_spread, zstep/2, 
                             sink, final_height, r*2, cutoff=0.9)
    spire_lines += make_spire(base_theta, dtheta/2, n_lines//2, line_spread, zstep/2, 
                              sink, final_height, r*2, cutoff=0.9)
    spire_lines += make_spire(base_theta, dtheta, n_lines, line_spread*2, zstep, sink, final_height, r)
    for spire_line in spire_lines:
        lines.append(cam.imageFromSpace(spire_line).flatten())
        
        
    for line in lines:
        canvas.append(draw.Lines(*line, close=False, fill='none', stroke=line_color))
    for ring in rings:
#         continue
        canvas.append(draw.Lines(*ring, close=True, fill='none', stroke=line_color))
    display(canvas)
    
def make_spire(base_theta, dtheta, n_lines, line_spread, zstep, sink, final_height, r, cutoff=2):
    spire_lines = [[] for _ in range(n_lines)]
    for z in np.arange(0, final_height, zstep):
        if z > final_height * cutoff:
            break
        zoop = z/final_height * 0.75 * pi
        curr_r = cos(zoop - pi/4) * r/4
        for i in range(n_lines):
            point = [
                curr_r * cos(base_theta + i * line_spread),
                curr_r * sin(base_theta + i * line_spread),
                z + sink
            ]
            spire_lines[i].append(point)
        base_theta += dtheta
    return spire_lines
    

In [16]:
dome(layers=20, n_lines=21, squash=1, noise_r=2, noise_coeff=0.005, bg_color="black", line_color="white")

In [14]:

gen = OpenSimplex(seed=123)
default_cam = ct.Camera(ct.RectilinearProjection(image=(800, 800)),
                        ct.SpatialOrientation(pos_y_m=-5000, elevation_m=-1800, tilt_deg=115, roll_deg=-40))
def dome(r=240, n=240, squash=0.6, layers=40, cam=default_cam, n_lines = 39, inset=0.04,
        tdiff=3, line_l=2.9, n_tent=0, noise_r=1, noise_coeff=1, bg_color='black', line_color="#3366ff"):
    random.seed(123)
    canvas = draw.Drawing(800, 800, center='origin', displayInline=False)
    canvas.append(draw.Rectangle(0, 0, 800, 800, fill=bg_color))
    rings = []
    lines = []
    height = r * 1.34 * squash
    
    for z in np.arange(0, height, height/layers):
        ring = []
        layer_r = r*cos(asin((r-z/squash)/r))
        if layer_r < 1e-5:
            continue
        for i in range(n):
            theta = i/n*2*pi
            ring.append([
                layer_r * cos(theta),
                layer_r * sin(theta),
                z
            ])
        rings.append(cam.imageFromSpace(ring).flatten())
        special_layer_r = layer_r
    
    offsets = []
    for _ in range(n_lines):
        offsets.append(random.uniform(1/tdiff, 1))
    total = sum(offsets)
    curr, thetas = 0, []
    for o in offsets:
        thetas.append(curr * 2 * pi)
        curr += o / total
    
    inner_height = r * (1.34 * (1 - 4*inset)) * squash
    for j, theta in enumerate(thetas):
        gen = OpenSimplex(seed=123+j)
        line = []
        rs = []
        line_step = inner_height/n *  2
        for z in np.arange(0, inner_height, line_step):
            sink = max(20 - z, 0) **2 * .1
            layer_r = r*cos(asin((r-z/squash)/r))
            rs.append(layer_r)
            line.append([
                layer_r * cos(theta) * (1 - inset),
                layer_r * sin(theta) * (1 - inset),
                z + height * inset + sink
            ])
        dr = rs[-1] - rs[-2]
        lol = random.uniform(1, 1.1)
        zs = np.arange(inner_height, lol*inner_height * line_l, line_step)
        steps = len(zs)
        target = 0.5
        for i, z in enumerate(zs):
            interp = i / steps
            layer_r += (dr * (1 - interp) + target * interp) * line_step
            line.append([
                layer_r * cos(theta) * (1 - inset),
                layer_r * sin(theta) * (1 - inset),
                z + height * inset + sink
            ])
            theta += gen.noise2d(noise_r*cos(theta), noise_r*sin(theta)) * interp * noise_coeff
            theta %= 2*pi
        
        lines.append(cam.imageFromSpace(line).flatten())
    n_tent = 0
    if n_tent:
        tent_gen = OpenSimplex(seed=123 + t)
        dr = rs[-1] - rs[-2]
        target = 20
        zs = np.arange(z + line_step * 4 + 30, inner_height * 3, line_step*8)
        maxi = len(zs)
        base_theta_offset = theta_offset
        layer_r = special_layer_r + 10
        for i, z in enumerate(zs):
            theta_offset = base_theta_offset + tent_gen.noise2d(0, 0.1*i) * 0.04
            linterp = i / maxi
            layer_r += min((linterp*target + (1-linterp)*dr)*line_step, 2) - 0.4
            a = [
                layer_r * cos(theta - theta_offset),
                layer_r * sin(theta - theta_offset),
                z + 20
            ]
            
            b = [
                layer_r * cos(theta + theta_offset),
                layer_r * sin(theta + theta_offset),
                z + 20
            ]
            lines.append(cam.imageFromSpace([a,b]).flatten())
            
    dtheta = 0.3
    base_theta = 0
    n_lines = 5
    line_spread = 0.18
    zstep = 10
    sink = 40
    final_height = height * 4
    spire_lines = make_spire(base_theta+pi, dtheta/3.5, n_lines//2, line_spread, zstep/2, 
                             sink, final_height, r*2, cutoff=0.9)
    spire_lines += make_spire(base_theta, dtheta/3.5, n_lines//2, line_spread, zstep/2, 
                              sink, final_height, r*2, cutoff=0.9)
    spire_lines += make_spire(base_theta, dtheta, n_lines, line_spread*2, zstep, sink, final_height, r)
    for spire_line in spire_lines:
        lines.append(cam.imageFromSpace(spire_line).flatten())
        
        
    for line in lines:
        canvas.append(draw.Lines(*line, close=False, fill='none', stroke=line_color))
    for ring in rings:
#         continue
        canvas.append(draw.Lines(*ring, close=True, fill='none', stroke=line_color))
    display(canvas)
    
def make_spire(base_theta, dtheta, n_lines, line_spread, zstep, sink, final_height, r, cutoff=2):
    spire_lines = [[] for _ in range(n_lines)]
    for z in np.arange(0, final_height, zstep):
        if z > final_height * cutoff:
            break
        zoop = z/final_height * 0.75 * pi
        curr_r = cos(zoop - pi/4) * r/4
        for i in range(n_lines):
            point = [
                curr_r * cos(base_theta + i * line_spread),
                curr_r * sin(base_theta + i * line_spread),
                z + sink
            ]
            spire_lines[i].append(point)
        base_theta += dtheta
    return spire_lines
dome(layers=10, n_lines=21, squash=1, noise_r=2, noise_coeff=0.005)

In [15]:
dome(layers=20, n_lines=21, squash=1, noise_r=2, noise_coeff=0.005, bg_color="black", line_color="white")